#**OBJECT DETECTION USING YOLOv8**

**Objective:**

The goal of this project is to perform real-time object detection on a 30-second dashboard camera view video using YOLOv8 mode.

This system identifies and localizes road objects—such as trucks, buses,Person and other vehicles—by drawing bounding boxes and adding class labels directly onto the video frames.

This project showcases an industry-standard, deep learning-based solution for traffic surveillance and smart mobility applications

In [ ]:
# Installing Ultralytics Framework (YOLOv8)
!pip install ultralytics --quiet

# Import Required Libraries
from ultralytics import YOLO
from google.colab import drive # Due to memory constraints i've used my Google Drive to access the files
import cv2,os
from IPython.display import Video, display

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Mouning Google Drive
drive.mount('/content/drive')

In [ ]:
# Defining a path to access the file which im going to annotate
input_video_path = "/content/drive/MyDrive/PROJECT/video1.mp4"

In [ ]:
# Defining a path to store my output
output_dir = "/content/drive/MyDrive/PROJECT"
os.makedirs(output_dir, exist_ok=True)
output_video = os.path.join(output_dir, "annotated_video.mp4")

In [ ]:
# Load YOLOv8 model
model = YOLO("yolov8n.pt") #Used YOLOv8n (nano version) due to RAM constraints. We can also use yolov8m.pt for better accuracy if system resources allow.

In [ ]:
# Open video reader and writer
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

In [ ]:
# Streaming inference: low memory usage
results = model.predict(source=input_video_path, stream=True)

for r in results:  # each r is a single frame result
    frame = r.orig_img  # original frame as numpy array
    boxes = r.boxes.xyxy.numpy()  # bounding box coordinates: [x1, y1, x2, y2]
    classes = r.boxes.cls.numpy().astype(int)  # detected class indices
    scores = r.boxes.conf.numpy()  # confidence scores

    # Overlay bounding boxes and labels
    for (x1, y1, x2, y2), cls, conf in zip(boxes, classes, scores):
        label = f"{model.names[cls]} {conf:.2f}"
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        cv2.putText(frame, label, (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    out.write(frame)  # save annotated frame

cap.release()
out.release()

In [ ]:
# Display result
display(Video(output_video, embed=True)) # The Annoted video automatically saved in our output path which we have chosen.